<a href="https://colab.research.google.com/github/ayushs0911/OpenCV/blob/main/Haar_Cascade_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Our Setup, Import Libaries, Create our Imshow Function and Download our Images
import cv2
import numpy as np
from matplotlib import pyplot as plt

# Define our imshow function 
def imshow(title = "Image", image = None, size = 10):
    w, h = image.shape[0], image.shape[1]
    aspect_ratio = w/h
    plt.figure(figsize=(size * aspect_ratio,size))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.show()

# Download and unzip our images and Haarcascade Classifiers
!wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/images.zip
!wget https://moderncomputervision.s3.eu-west-2.amazonaws.com/haarcascades.zip

!unzip -qq images.zip
!unzip -qq haarcascades.zip

## **HAAR Cascade Classifiers**

Developed by Viola and Jones in 2001.

An object detection method that uses a series of classifiers (cascade) to identify objects in an image. They are trained to identify one type of object, however, we can use several of them in parallel e.g. detecting eyes and faces together. HAAR Classifiers are trained using lots of positive images (i.e. images with the object present) and negative images (i.e. images without the object present).
![](https://github.com/rajeevratan84/ModernComputerVision/raw/main/haar.png)


The Haar cascade classifier is an object detection algorithm used for identifying objects or specific features within images or video frames. It is particularly known for its effectiveness in detecting faces, although it can be trained to detect other objects as well.

The algorithm is based on Haar-like features, which are simple rectangular filters that are applied to different regions of an image. These features are computed by calculating the difference between the sum of pixel intensities in the white and black rectangles within the region of interest. By evaluating a large number of these features at various scales and positions, the Haar cascade classifier can effectively differentiate between the object of interest and the background.

The Haar cascade classifier employs a cascade of classifiers, which is a sequence of stages where each stage consists of multiple weak classifiers. These weak classifiers are simple classifiers, such as decision trees or linear classifiers, that are combined to form a strong classifier capable of making accurate detections. The cascade structure allows for fast and efficient object detection by quickly rejecting regions that are unlikely to contain the object, thus reducing the computational load.

The training of a Haar cascade classifier involves a two-step process: positive sample collection and negative sample collection. Positive samples are images containing the object of interest, while negative samples are images without the object. The algorithm is trained using a machine learning technique called AdaBoost (Adaptive Boosting), which iteratively combines weak classifiers and adjusts their weights to improve overall detection performance.

Once the Haar cascade classifier is trained, it can be used to detect the object in new images or video frames. The classifier slides a detection window across the image at different scales and positions, applying the learned cascade of classifiers to determine if the object is present. If a detection threshold is exceeded, the object is considered to be present in that region.

While Haar cascade classifiers have been widely used and have shown good performance for certain object detection tasks, they have some limitations. They may struggle with detecting objects under variations in lighting conditions, scale, orientation, or occlusion. Additionally, they may produce false positives or false negatives depending on the quality of training data and the chosen detection thresholds.

More advanced object detection techniques, such as deep learning-based approaches (e.g., convolutional neural networks), have gained popularity due to their superior performance on a wide range of object detection tasks. Nonetheless, Haar cascade classifiers remain relevant and can be a lightweight option for certain applications or scenarios with specific requirements.

In [ ]:
#point opencv's Cascade classifier function to where our classifier is stored 
face_classifier = cv2.CascadeClassifier("/content/Haarcascades/haarcascade_frontalface_default.xml")

# load image and convert to grayscale 
image = cv2.imread('./images/Trump.jpg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


#classifier returns the ROI of the detected face as a tuple 
# it stores the top left coordinate and the bottom right coordinates 
faces = face_classifier.detectMultiScale(gray, scaleFactor = 1.3, minNeighbors = 5)

#when no face detected, face_classifier returns empty tuple 
if faces is ():
  print("No faces found")

#we iterate through our faces array and draw a rectangle over each face in frame
for (x, y, w, h) in faces:
  cv2.rectangle(image, (x,y), (x+w, y+h), (127, 0, 255), 2)

imshow('Face detection', image)

## **Simple Eye & Face Detection using Haarcascade Classifiers**

In [ ]:
import numpy as np
import cv2
 
face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_eye.xml')
 
img = cv2.imread('images/Trump.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_classifier.detectMultiScale(gray, 1.3, 5)

# When no faces detected, face_classifier returns and empty tuple
if faces is ():
    print("No Face Found")

for (x, y, w, h) in faces:
  cv2.rectangle(img, (x,y), (x+w, y+h), (127,0,255),2)
  roi_gray = gray[y:y+h, x:x+w]
  roi_color = img[y:y+h, x:x+w]
  eyes = eye_classifier.detectMultiScale(roi_gray, 1.2, 3)
  for (ex, ey, ew, eh) in eyes:
    cv2.rectangle(roi_color, (ex,ey), (ex+ew, ey+eh), (255,255,0), 2)

imshow("Eye and Face detection", img)


## **Using Colab's Code Snippets let's access the webcam for an input**

Note: Requires your computer to have a webcam

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
import numpy as np
import cv2
 
face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_eye.xml')
 
img = cv2.imread('photo.jpg')
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

faces = face_classifier.detectMultiScale(gray, 1.3, 5)

# When no faces detected, face_classifier returns and empty tuple
if faces is ():
    print("No Face Found")

for (x,y,w,h) in faces:
    cv2.rectangle(img,(x,y),(x+w,y+h),(127,0,255),2)
    roi_gray = gray[y:y+h, x:x+w]
    roi_color = img[y:y+h, x:x+w]
    eyes = eye_classifier.detectMultiScale(roi_gray)
    for (ex,ey,ew,eh) in eyes:
        cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(255,255,0),2)

imshow('Eye & Face Detection',img)

### **Bonus Code - Use your webcam to do live face and eye detection**

This only works on a local machine, will not work in Colab

In [ ]:
import cv2
import numpy as np

face_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_frontalface_default.xml')
eye_classifier = cv2.CascadeClassifier('Haarcascades/haarcascade_eye.xml')

def face_detector(img, size=0.5):
    # Convert image to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    if faces is ():
        return img
    
    for (x,y,w,h) in faces:
        x = x - 50
        w = w + 50
        y = y - 50
        h = h + 50
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_classifier.detectMultiScale(roi_gray)
        
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,0,255),2) 
            
    roi_color = cv2.flip(roi_color,1)
    return roi_color

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    cv2.imshow('Our Face Extractor', face_detector(frame))
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      